In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense,  Dropout, Input, LSTM, Embedding,SpatialDropout1D, BatchNormalization, Flatten, LayerNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow_text as text

import pandas_tfrecords as pdtfr

import matplotlib.pyplot as plt
import re

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn
from nltk import word_tokenize,pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict        
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pathlib import Path


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import pickle
from sklearn.metrics import confusion_matrix

import keras_tuner as kt
import seaborn as sns
import tensorflow_datasets as tfds


tf.random.set_seed(0)
np.random.seed(0)
stop_words_list = stopwords.words('english') 

#tf.config.run_functions_eagerly(True)
import tensorflow_data_validation as tfdv
#tf.debugging.set_log_device_placement(True)



2024-05-14 11:07:04.558668: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 11:07:04.580261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 11:07:04.580292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 11:07:04.581019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 11:07:04.585204: I tensorflow/core/platform/cpu_feature_guar

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11743047593946994537
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1137967104
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15886912803736253735
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [2]:
# print(tf.__version__)
# import tensorflow_hub as hub

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#         tf.config.experimental.set_memory_growth(gpus[0], True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         print(e)



Num GPUs Available:  1


2024-05-14 11:07:06.720374: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 11:07:06.720488: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 11:07:06.720532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Read dataset


In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True, 
                          as_supervised=True)

train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset

2024-05-14 11:07:06.732270: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 11:07:06.732587: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-14 11:07:06.732659: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
for i, o in train_dataset.take(1):
    print(i.shape, o.shape)
    print(i, o)

() ()
tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) tf.Tensor(0, shape=(), dtype=int64)


2024-05-14 11:07:06.863892: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [5]:
def convert_tfdataframe_to_pddataframe(tf_df):
    data_list = []
    for i, (features, label) in enumerate(tf_df):
        
        if i % 5000 == 0 : print(f"{i}/{len(tf_df)}")
        
        data_list.append((features.numpy().decode('utf-8'), label.numpy()))

    # Create a Pandas DataFrame
    return pd.DataFrame(data_list, columns = [ "review", "sentiment"])

# Create a Pandas DataFrame
train_dataset = convert_tfdataframe_to_pddataframe(train_dataset)
train_dataset

0/25000
5000/25000
10000/25000
15000/25000
20000/25000


,review,sentiment
0,This was an absolutely terrible movie. Don't b...,0
1,"I have been known to fall asleep during films,...",0
2,Mann photographs the Alberta Rocky Mountains i...,0
3,This is the kind of film for a snowy Sunday af...,1
4,"As others have mentioned, all the women that g...",1
...,...,...
24995,"I have a severe problem with this show, severa...",0
24996,"The year is 1964. Ernesto ""Che"" Guevara, havin...",1
24997,Okay. So I just got back. Before I start my re...,0
24998,When I saw this trailer on TV I was surprised....,0


In [6]:
# Create a Pandas DataFrame
test_dataset = convert_tfdataframe_to_pddataframe(test_dataset)
test_dataset

0/25000
5000/25000
10000/25000
15000/25000
20000/25000


,review,sentiment
0,There are films that make careers. For George ...,1
1,"A blackly comic tale of a down-trodden priest,...",1
2,"Scary Movie 1-4, Epic Movie, Date Movie, Meet ...",0
3,Poor Shirley MacLaine tries hard to lend some ...,0
4,As a former Erasmus student I enjoyed this fil...,1
...,...,...
24995,"Feeling Minnesota is not really a road movie, ...",0
24996,"This is, without doubt, one of my favourite ho...",1
24997,Most predicable movie I've ever seen...extreme...,0
24998,It's exactly what I expected from it. Relaxing...,1


# Check statistics

In [7]:
train_dataset.dtypes

review       object
sentiment     int64
dtype: object

In [8]:
# Instantiate a StatsOptions class and define the feature_allowlist property
stats_options = tfdv.StatsOptions(
    feature_allowlist=train_dataset.columns.tolist(),
    enable_semantic_domain_stats= True # Because we are processing text
    )
stats_options

<_generators=None, _feature_allowlist=['review', 'sentiment'], _schema=None, label_feature=None, weight_feature=None, _slice_functions=None, _sample_rate=None, num_top_values=20, frequency_threshold=1, weighted_frequency_threshold=1.0, num_rank_histogram_buckets=1000, _num_values_histogram_buckets=10, _num_histogram_buckets=10, _num_quantiles_histogram_buckets=10, epsilon=0.01, infer_type_from_schema=False, _desired_batch_size=None, enable_semantic_domain_stats=True, _semantic_domain_stats_sample_rate=None, _per_feature_weight_override=None, _vocab_paths=None, _add_default_generators=True, _use_sketch_based_topk_uniques=False, _slice_sqls=None, _experimental_num_feature_partitions=1, _experimental_result_partitions=1, _slicing_config=None, _experimental_filter_read_paths=False, _per_feature_stats_config=None>

In [9]:
train_stats = tfdv.generate_statistics_from_dataframe(train_dataset, stats_options) 

# get the number of features used to compute statistics
print(f"Number of features used: {len(train_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {train_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {train_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {train_stats.datasets[0].features[-1].path.step[0]}")
tfdv.visualize_statistics(train_stats)


Number of features used: 2
Number of examples used: 25000
First feature: review
Last feature: sentiment


# Set Parameters

In [10]:
emb_dim=200
batchsize=256

sequence_size= 500
READ_DATA = True
ETL_STOPWORDS= True
ETL_SENTENCE_SIZE = True
TUNNING = True
TRAIN_MODE= True
INFERENCE_MODE = True

max_trials = 30
threshold = 1
classN=1
min_freq = 5



In [11]:

DATA_CHECKPOINTS_DIR = './checkpoints'
tfrecord_filename = "train_tmp.tfrecord"
train_tmp_record_path = f'{DATA_CHECKPOINTS_DIR}/{tfrecord_filename}'

!mkdir -p {DATA_CHECKPOINTS_DIR}



# Set Functions to perform data cleaning and feature engineering

In [12]:
def check_word_frequency(train_df):
    
    print("Dataset lentgh : ",len(train_df))

    tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;=?@[\\]^_´`«»{|}~\t\n\'',oov_token='<oov>')# sequence_size = 500 ~= mean+ std sentence size
    tokenizer.fit_on_texts(train_df.review.values.tolist())

    word_frequency = {}
    for i, (word, count) in enumerate(tokenizer.word_counts.items()):
        word_frequency[word]=count
            
    return word_frequency


def update_stop_words(word_frequency, freq,max_freq,stop_word):
 
    it = 0
    for word, count in word_frequency.items():
        if count < freq:
            if it < 10:
                print(word)
                it +=1
            stop_word.append(word)
        elif count >max_freq :
            stop_word.append(word)
            
    return list(set(stop_word))


word_frequency= check_word_frequency(train_dataset)
max_freq = np.max(list(word_frequency.values())) +1 
# Update stop words list with the less frequent 
updated_stop_word = update_stop_words(word_frequency, min_freq,max_freq, stop_words_list.copy())

Dataset lentgh :  25000
ironside.
columbian
revolutions.
sette
accord
gunfighters
west.<br
>canadian
paddle
libido.


In [13]:

def create_trainingdev_set(sequences_aux,df_aux,maxlen,devtest_set):
    x = pad_sequences(sequences_aux, padding='post', maxlen=maxlen)
    y = np.asarray(df_aux.sentiment.values.tolist()).astype('float32')
    inputs_train = []
    inputs_val = []
    if devtest_set:

        test_elements_index = np.random.choice(len(x), int(len(x)*0.50), replace=False) 
        print("devtest_elements_index length : ",len(test_elements_index))
        
        x_in = np.take(x, test_elements_index, 0)
        y_in = np.take(y, test_elements_index, 0)
        print("x_in shape", x_in.shape)
        print("y_in shape", y_in.shape)

        x_test= np.delete(x, test_elements_index, 0)
        y_test= np.delete(y, test_elements_index, 0)
        print("x_test shape", x_test.shape)
        print("y_test shape", y_test.shape)
        
        inputs_train.append(x_in)
        inputs_val.append(x_test)
        
        return inputs_train,y_in, inputs_val,y_test 
    
    else:
        print("x shape", x.shape)
        print("y shape", y.shape)
        
        inputs_train.append(x)
        
        
        return inputs_train,y  

In [20]:


def limit_words(review, sentiment):
    # Split the string into tokens
    tokens = tf.strings.split(review)

    # Limit the number of tokens to max_words
    limited_tokens = tokens[:sequence_size]
    
    # Join the tokens back into a string
    limited_review = tf.strings.reduce_join(limited_tokens, separator=' ')
        
    return limited_review, sentiment


def save_record(dataset):

    with tf.io.TFRecordWriter(train_tmp_record_path) as writer:
        for review, sentiment in dataset:
            
            # Serialize example
            feature = {
                'review': tf.train.Feature(bytes_list=tf.train.BytesList(value=[review.numpy()])),
                'sentiment': tf.train.Feature(int64_list=tf.train.Int64List(value=[sentiment.numpy()]))
            }
            example_proto = tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()
            
            writer.write(example_proto)
            
def clean_reviews(review, sentiment):
    
    lowercase = tf.strings.lower(review)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    
    tokens = tf.strings.split(stripped_html)
    # Filter out words to remove
    words_to_remove_tensor = tf.constant(updated_stop_word)
    mask = tf.logical_not(tf.reduce_any(tf.equal(tokens, words_to_remove_tensor[:, tf.newaxis]), axis=0))
    
    # Filter out words to remove
    filtered_tokens = tf.boolean_mask(tokens, mask)
    
    # Join the remaining tokens back into a string
    processed_data = tf.strings.reduce_join(filtered_tokens, separator=' ')
    
    sentence_size = len(tf.strings.split(processed_data))
    return (processed_data, sentiment , sentence_size)


def process_stop_words(td_dataset):

    # Convert to tf.dataset
    td_dataset = tf.data.Dataset.from_tensor_slices((td_dataset['review'], td_dataset['sentiment']))

    for i, o in td_dataset.take(1):
        print(i.shape, o.shape)
        print("Pre processed ds : ",i, o )
   
    
    td_dataset = td_dataset.map(clean_reviews, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    for i, o, l in td_dataset.take(3):
        print("Post processed ds : ",l, o, i )
        
    data_list = []
    for i, (features, label, size) in enumerate(td_dataset):
        
        if i % 5000 == 0 : print(f"{i}/{len(td_dataset)}")
        
        data_list.append((features.numpy(), label.numpy(), size.numpy()))

    # Create a Pandas DataFrame
    return pd.DataFrame(data_list, columns = [ "review", "sentiment"])

train_ds = process_stop_words(train_dataset)
train_ds

() ()
Pre processed ds :  tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) tf.Tensor(0, shape=(), dtype=int64)


TypeError: process_stop_words.<locals>.<lambda>() missing 1 required positional argument: 'y'

In [ ]:
aaaaaaaaaaaa

In [ ]:
test_ds  = process_stop_words(test_dataset)

In [ ]:
  
aaaaaaaaaaaaaaaaa
if ETL_STOPWORDS:
    # Get word frequency from tokenizer
    word_frequency= check_word_frequency(train_dataset)
    max_freq = np.max(list(word_frequency.values())) +1 
    
    # Update stop words list with the less frequent 
    updated_stop_word = update_stop_words(word_frequency, min_freq,max_freq, stop_words_list.copy())
    
    # Convert to tf.dataset
    train_ds = tf.data.Dataset.from_tensor_slices(( train_dataset['review'].values,  train_dataset['sentiment'].values))
    #test_ds = tf.data.Dataset.from_tensor_slices((test_dataset['review'].values, test_dataset['sentiment'].values))

    
    for i, o in train_ds.take(3):
        print(i.shape, o.shape)
        print("Train ds : ",i, o )
    
    # for i, o in test_ds.take(3):
    #     print("Test ds : ",i, o)

    
    # Clean dataset and calculated the sentence length
    #with tf.device("/gpu:0"):  

    train_ds = train_ds.map(clean_reviews, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    #test_ds = test_ds.map(clean_reviews, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    for i, o, l in train_ds.take(3):
        print("Post cleaning train ds : ",l.numpy(), o.numpy(), i.numpy() )
    
    # for i, o, l in test_ds.take(3):
    #     print("Post cleaning test ds : ",l.numpy(), o.numpy(), i.numpy() )
    
    print(train_ds)

In [ ]:
sentence_data = []
for i, (features, label, sentence_size)  in enumerate(train_ds):
    if i % 1000 == 0 : print(f"{i}/{len(train_ds)}")
    sentence_data.append(sentence_size.numpy())

In [ ]:
if ETL_STOPWORDS:aaaaaaaaaa
   
    
    #print("Converting to dataframe ..")
    #train_ds = train_ds.cache()#.prefetch(tf.data.experimental.AUTOTUNE)
    #test_ds = test_ds.cache()#.prefetch(tf.data.experimental.AUTOTUNE)
    #train_ds =  tfds.as_dataframe(train_ds, info) 
    #test_ds =  tfds.as_dataframe(test_ds, info) 
    train_ds =  convert_tfdataframe_to_pddataframe(train_ds)#tfds.as_dataframe(train_ds, info) 
    test_ds =  convert_tfdataframe_to_pddataframe(test_ds)#tfds.as_dataframe(train_ds, info) 

    print("Creating the tokenizer ...")
    tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;=?@[\\]^_´`«»{|}~\t\n\'',oov_token='<oov>')# sequence_size = 500 ~= mean+ std sentence size
    tokenizer.fit_on_texts(train_ds.review.values.tolist())
    
    with open(f'{DATA_CHECKPOINTS_DIR}/word_frequency.pkl', 'wb') as f:
        pickle.dump(word_frequency, f)

    with open(f'{DATA_CHECKPOINTS_DIR}/updated_stop_word.pkl', 'wb') as f:
        pickle.dump(updated_stop_word, f)
        
    with open(f'{DATA_CHECKPOINTS_DIR}/tokenizer.pkl', 'wb') as f:
        pickle.dump(tokenizer, f)
        
    train_ds.to_pickle(f"{DATA_CHECKPOINTS_DIR}/train_ds.pkl")  
    test_ds.to_pickle(f"{DATA_CHECKPOINTS_DIR}/test_ds.pkl")  
    
else:
    
    with open(f'{DATA_CHECKPOINTS_DIR}/word_frequency.pkl', 'rb') as f:
        word_frequency = pickle.load(f)
        
    with open(f'{DATA_CHECKPOINTS_DIR}/updated_stop_word.pkl', 'rb') as f:
        updated_stop_word = pickle.load(f)
        
    with open(f'{DATA_CHECKPOINTS_DIR}/tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)

        
    train_ds = pd.read_pickle(f"{DATA_CHECKPOINTS_DIR}/train_ds.pkl")  
    test_ds = pd.read_pickle(f"{DATA_CHECKPOINTS_DIR}/test_ds.pkl")    
    
    

# Check sentence size
 

In [ ]:
    
    # train_ds = train_ds.map(limit_words, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # test_ds = test_ds.map(limit_words, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# Get the sentence size
train_ds["sentence_size"] = train_ds.review.apply(lambda  x : len(x.split()))
train_ds["sentence_size"] = train_ds.review.apply(lambda  x : len(x.split()))

sns.boxplot(x=test_ds['sentence_size'])
train_dataset.sentence_size.describe()

In [ ]:
train_ds[train_ds.sentence_size > 800].review.values[0]

In [ ]:
sns.boxplot(x=test_ds['sentence_size'])
test_ds.sentence_size.describe()

Finding the outliers to fix the sentence size . Every sample with size considered as outlier will be trimmed

In [ ]:

Q1_train = train_ds.sentence_size.quantile(0.25)
Q3_train = train_ds.sentence_size.quantile(0.75)
IQR_train = Q3_train - Q1_train

print("Train IQR : ",IQR_train)
print("Final training dataset size : ",(Q1_train - 1.5 * IQR_train),(Q3_train + 1.5 * IQR_train), len(train_dataset[ train_dataset.sentence_size <= (Q3_train + 1.5 * IQR_train)]))
print("Training data to be trimed : ",len(train_dataset[ train_dataset.sentence_size > (Q3_train + 1.5 * IQR_train)]))


In [ ]:
Q1_test = test_ds.sentence_size.quantile(0.25)
Q3_test = test_ds.sentence_size.quantile(0.75)
IQR_test = Q3_test - Q1_test

print("Test IQR : ",IQR_test)
print("Final testing dataset size : ",(Q1_test - 1.5 * IQR_test),(Q3_test + 1.5 * IQR_test), len(test_dataset[ test_dataset.sentence_size <= (Q3_test + 1.5 * IQR_test)]))
print("Testing data to be trimed : ",len(test_dataset[ test_dataset.sentence_size > (Q3_test + 1.5 * IQR_test)]))


Before thinking on trimming, we must check if any important sentiment information will not be deleted.

# Tokenize and prepare embeddings

In [ ]:
from transformers import  RobertaTokenizer, TFRobertaModel
aaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaa

num_labels = len(test_dataset.sentiment.unique())  # number of labels for your classification task
model_name = 'roberta-base'  # or specify the path to a custom model
tokenizer = RobertaTokenizer.from_pretrained(model_name)
encoder = TFRobertaModel.from_pretrained(model_name)
encoder.trainable = False

max_length = pickle.load(open(sequence_size,"rb") )
n_utterance_history = pickle.load(open(n_utterance_history_key,"rb") )

n_utterance_history, max_length

In [ ]:
IQR_total = np.minimum((Q3_train + 1.5 * IQR_train),(Q3_test + 1.5 * IQR_test))
sequence_size = int(IQR_total)
sequence_size

In [ ]:
if ETL_SENTENCE_SIZE == True :
    
    df = check_sentece_size(df,sequence_size)
    df_filtered=df.copy()
    df_filtered = df[df['sentence_size'] > 0] 
    df_filtered = df_filtered.reset_index().drop(["index"], axis = 1)
    
    test_dataset = check_sentece_size(test_dataset,sequence_size)
    test_dataset_filtered=test_dataset.copy()
    test_dataset_filtered = test_dataset[test_dataset['sentence_size'] > 0] 
    test_dataset_filtered = test_dataset_filtered.reset_index().drop(["index"], axis = 1)
    
    
    tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;=?@[\\]^_´`«»{|}~\t\n\'',oov_token='<oov>')# sequence_size = 500 ~= mean+ std sentence size
    tokenizer.fit_on_texts(df_filtered.review.values.tolist())
    tokenizer.fit_on_texts(test_dataset_filtered.review.values.tolist())
    word_frequency= check_word_frequency(tokenizer)
    
    with open(f'{DATA_CHECKPOINTS_DIR}/word_frequency.pkl', 'wb') as f:
        pickle.dump(word_frequency, f)

    with open(f'{DATA_CHECKPOINTS_DIR}/tokenizer.pkl', 'wb') as f:
        pickle.dump(tokenizer, f)
        
    df.to_pickle(f"{DATA_CHECKPOINTS_DIR}/df.pkl")  
    test_dataset.to_pickle(f"{DATA_CHECKPOINTS_DIR}/test_dataset.pkl")  
    
    df_filtered.to_pickle(f"{DATA_CHECKPOINTS_DIR}/df_filtered.pkl")  
    test_dataset_filtered.to_pickle(f"{DATA_CHECKPOINTS_DIR}/test_dataset_filtered.pkl") 
    
else:
    df = pd.read_pickle(f"{DATA_CHECKPOINTS_DIR}/df.pkl")
    test_dataset = pd.read_pickle(f"{DATA_CHECKPOINTS_DIR}/test_dataset.pkl")
    
    df_filtered = pd.read_pickle(f"{DATA_CHECKPOINTS_DIR}/df_filtered.pkl")
    test_dataset_filtered = pd.read_pickle(f"{DATA_CHECKPOINTS_DIR}/test_dataset_filtered.pkl")
    
    with open(f'{DATA_CHECKPOINTS_DIR}/word_frequency.pkl', 'rb') as f:
        word_frequency = pickle.load(f)
        
    with open(f'{DATA_CHECKPOINTS_DIR}/tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
        


In [ ]:

plt.figure()
pd.Series(df_filtered.sentence_size).hist()
plt.title("Sentence Size for training dataset")

plt.figure()
pd.Series(df_filtered.sentiment).hist()
plt.title("Sentiment for training dataset")

plt.show()
print(pd.Series(df_filtered.sentence_size).describe())

plt.figure()
pd.Series(test_dataset_filtered.sentence_size).hist()
plt.title("Sentence Size for test dataset")

plt.figure()
pd.Series(test_dataset_filtered.sentiment).hist()
plt.title("Sentiment for test dataset")

plt.show()
print(pd.Series(test_dataset_filtered.sentence_size).describe())

# Deep learning approach

In [ ]:
df_filtered.sentiment.unique()

In [ ]:
df_filtered = (df_filtered.dropna()).reset_index().drop(["index"], axis = 1)
test_dataset_filtered = (test_dataset_filtered.dropna()).reset_index().drop(["index"], axis = 1)
test_dataset_filtered

In [ ]:
ft = fasttext.load_model('../cc.en.300.bin')
fasttext.util.reduce_model(ft, emb_dim)
ft.get_dimension()

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;=?@[\\]^_´`«»{|}~\t\n\'',oov_token='<oov>')# sequence_size = 500 ~= mean+ std sentence size
tokenizer.fit_on_texts(df_filtered.review.values.tolist())
tokenizer.fit_on_texts(test_dataset_filtered.review.values.tolist())
_, vocab_len,sequences,word_index= tokenize_data(tokenizer, df_filtered)
print("Train Vocab length : It should be the complete one : ",vocab_len)
_, vocab_len_test,sequences_test,word_index_test= tokenize_data(tokenizer, test_dataset_filtered)
print("Test Vocab length. It should be the complete one : ",vocab_len_test)

In [ ]:
embeddings_matrix=get_embeddings_fb(ft,word_index,emb_dim,vocab_len)
print(embeddings_matrix)

In [ ]:
x, y = create_trainingdev_set(sequences,df_filtered,sequence_size, False)
x_dev,y_dev,x_test,y_test = create_trainingdev_set(sequences_test,test_dataset_filtered,sequence_size,True)
y_dev

# Start training process

### Run training process 


In [ ]:


def get_model_dynamic(best_hps):
    pr = tf.keras.metrics.AUC(curve="PR")
    p = tf.keras.metrics.Precision()
    r = tf.keras.metrics.Recall()

    inp = Input(batch_shape = (None,sequence_size,), name="X")

    ft = fasttext.load_model('../cc.en.300.bin')
    fasttext.util.reduce_model(ft, best_hps["emb_dim"])
    embeddings_matrix_tmp=get_embeddings_fb(ft,word_index,best_hps["emb_dim"],vocab_len)

    X = Embedding(input_dim = vocab_len, output_dim = best_hps["emb_dim"], input_length = sequence_size ,name ="emb_layer",  weights=[embeddings_matrix_tmp], trainable=False, mask_zero=True) (inp)# Use masking to handle the variable sequence lengths
    #X = SpatialDropout1D(best_hps.get('SpatialDropout1D_1'))(X)
    X = (LSTM(best_hps.get('units_0'),  dropout=best_hps.get('Dropout_0'), return_sequences=False))(X)

    X = BatchNormalization()(X)
    X = Dropout(best_hps.get('Dropout_1'))(X)
    X = Dense(best_hps.get('units_1'), activation='relu')(X)


    Z = Dense(classN, activation='sigmoid')(X)

    model =  Model(inputs = inp, outputs=Z )
    model.summary()

    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=best_hps.get('learning_rate')), metrics =[tf.keras.metrics.BinaryAccuracy(),pr,p,r,tf.keras.metrics.AUC()])

    return model

def get_model_tunning(hp):

    hp_units_0 = hp.Int('units_0', min_value=32, max_value=256, step=32)
    hp_units_1 = hp.Int('units_1', min_value=32, max_value=256, step=32)
    hp_drop_0 = hp.Choice('Dropout_0', values=[ 0.2, 0.3, 0.4, 0.5])
    hp_drop_1 = hp.Choice('Dropout_1', values=[ 0.2, 0.3, 0.4, 0.5])
    #hp_SpatialDropout1D_1 = hp.Choice('SpatialDropout1D_1', values=[0.1, 0.2, 0.3, 0.4, 0.5])
    hp_emb_dim = hp.Choice('emb_dim', values=[ 50, 100, 200])


    ft = fasttext.load_model('../cc.en.300.bin')
    fasttext.util.reduce_model(ft, hp_emb_dim)
    embeddings_matrix_tmp=get_embeddings_fb(ft,word_index,hp_emb_dim,vocab_len)


    inp = Input(batch_shape = (None,sequence_size,), name="X")
    
    X = Embedding(input_dim = vocab_len, output_dim = hp_emb_dim, input_length = sequence_size ,name ="emb_layer",  weights=[embeddings_matrix_tmp], trainable=False, mask_zero=True) (inp)# Use masking to handle the variable sequence lengths
    #X = SpatialDropout1D(hp_SpatialDropout1D_1)(X)
    X = (LSTM(hp_units_0, dropout=hp_drop_0, return_sequences=False))(X)

    X = BatchNormalization()(X)
    X = Dropout(hp_drop_1)(X)
    X = Dense(hp_units_1, activation='relu')(X)


    Z = Dense(classN, activation='sigmoid')(X)
    
    model =  Model(inputs = inp, outputs=Z )
    
    hp_learning_rate = hp.Choice('learning_rate', values=[0.001, 0.005])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer=Adam(learning_rate=hp_learning_rate), metrics =[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()])
    
    return model


tuner = kt.BayesianOptimization(
        get_model_tunning,
        objective= kt.Objective('val_loss', direction="min"), #kt.Objective('val_auc', direction="max"),# #val_binary_accuracy
        max_trials = max_trials,
        directory=r"Hyperparam_tunning",
        project_name='keras_tunning'
    )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)#val_auc
if TUNNING:
    tuner.search(x[0], y,  epochs=300, batch_size=batchsize, validation_data = (x_dev[0],y_dev), verbose =2, callbacks=[stop_early]) 

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps


In [ ]:
best_hps["units_0"],best_hps["units_1"],best_hps["Dropout_0"],best_hps["Dropout_0"],best_hps["learning_rate"]
#(64, 64, 0.2, 0.2, 0.5, 0.005)

In [ ]:
model_dynamic = get_model_dynamic(best_hps)

if FIT_MODEL:
    
    history = model_dynamic.fit(x[0], y, 
              batch_size=batchsize, 
              epochs=300,
              shuffle=True,
              validation_data = (x_dev[0],y_dev),
              callbacks=[PlotLossesCallback() ,stop_early]
             )
    
    model_dynamic.save(f'{DATA_CHECKPOINTS_DIR}_dynamic/model_dynamic')

    with open(f'{DATA_CHECKPOINTS_DIR}_dynamic/history.pickle', 'wb') as handle:
        pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

    


else:
    model_dynamic = tf.keras.models.load_model(f'{DATA_CHECKPOINTS_DIR}_dynamic/model_dynamic')

    with open(f'{DATA_CHECKPOINTS_DIR}_dynamic/history.pickle', 'rb') as handle:
        history = pickle.load(handle)
        plt.figure()
        plt.plot(history['binary_accuracy'])
        plt.plot(history['val_binary_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')

        plt.figure()
        plt.plot(history['loss'])
        plt.plot(history['val_loss'])
        plt.title('Model training loss')
        plt.ylabel('Loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')

        plt.show()

In [ ]:
yhat =model_dynamic.predict(x_test[0][0:100])
yhat[yhat >=0.5] = 1
yhat[yhat < 0.5] = 0 
yhat

y_test = y_test.reshape((len(y_test),1))
y_test
print("x 1º line : "+str(x))
print("y : ",y) 
print("y_test : ",y_test) 

In [ ]:

yhat =model_dynamic.predict(x_test)
yhat[yhat >=0.5] = 1
yhat[yhat < 0.5] = 0 
y_test = y_test.reshape((len(y_test),1))

pr = tf.keras.metrics.AUC(curve="PR")
print()

p = tf.keras.metrics.Precision()
p.update_state(y_test,yhat)

r = tf.keras.metrics.Recall()
r.update_state(y_test,yhat)

base_pr = pr(y_test, yhat).numpy()
base_p = p.result().numpy()
base_r = r.result().numpy()
print("AUC",base_pr," | Precision : ",base_p, " | Recall : ",base_r)


In [ ]:
confusion_matrix(list(y_test),list(yhat))

In [ ]:

base_tn, base_fp, base_fn, base_tp = confusion_matrix(list(y_test),list(yhat)).ravel()
base_tn, base_fp, base_fn, base_tp



In [ ]:
def print_predictions(X, pred):
    for i in range(len(X)):
        xx = [tokenizer.index_word.get(ind) for ind in X[i] if tokenizer.index_word.get(ind) is not None]
        print(' '.join(xx), "Prediction :", int(pred[i])," - Real :",y_test[i][0],"\n")
        
        if i==10:
            break

print_predictions(x_test[0], yhat)